In [7]:
# === 基本套件 ===
import os
import re
import time
from datetime import datetime

# === 數據處理 ===
import numpy as np
import pandas as pd
import pyautogui
from bs4 import BeautifulSoup

# === 瀏覽器自動化 ===
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

# === 特殊驅動 ===
import undetected_chromedriver as uc

In [5]:
def getinfor(代碼):
    """
    抓取股票資訊並計算相關指標
    Args:
        代碼: 股票代碼
    """
    # === 瀏覽器設定 ===
    service = Service(ChromeDriverManager().install())
    options = uc.ChromeOptions()
    # 設定瀏覽器選項
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-infobars')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    # === 抓取股利政策資料 ===
    driver = uc.Chrome(options=options)
    driver.get(f'https://goodinfo.tw/tw/StockDividendPolicy.asp?STOCK_ID={代碼}')
    time.sleep(5)
    driver.refresh()
    
    # 解析網頁內容
    page_source = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # 取得歷年平均殖利率
    share = soup.find('table', class_='b0v1h1 p4_2 row_mouse_over')
    for i in share:
        s = i.find('tr', align='center')
        td_tags = s.find_all('td')
        歷年平均殖利率 = float(td_tags[3].text)

    # === 處理歷年資料 ===
    year = []
    rate = []
    for j in range(1, 16):
        row = soup.find('tr', id=f'row{j}')
        if row:
            年份 = row.find_all('b')
            盈餘分配率 = row.find_all('td')
            if len(年份) > 2 and len(盈餘分配率) > 0:
                if ('20' in 年份[2].text and 'Q' not in 年份[2].text) or 'H' in 年份[2].text:
                    try:
                        year.append(年份[2].text)
                        rate.append(float(盈餘分配率[-1].text)/100)
                    except:
                        rate.append(0.00001)
    driver.quit()

    # === 資料處理與計算 ===
    # 建立基本DataFrame
    df = pd.DataFrame(year, columns=['年份'])
    df['盈餘分配率'] = rate

    # 處理半年報資料
    H = df[df['年份'].str.contains('H')]
    y = []
    r = []
    for i in range(len(H) - 1):
        if H['年份'].iloc[i][:2] == H['年份'].iloc[i + 1][:2]:
            y.append('20' + H['年份'].iloc[i][:2])
            r.append((H['盈餘分配率'].iloc[i] + H['盈餘分配率'].iloc[i + 1]) / 2)

    # 合併資料
    df1 = pd.DataFrame(y, columns=['年份'])
    df1['盈餘分配率'] = r
    df = df[~df['年份'].str.contains('H')]
    df = pd.concat([df, df1], ignore_index=False)
    df = df.sort_values(by='年份', ascending=False)

    # === 計算平均值 ===
    平均 = {}
    加權 = {}
    for i in [3, 5, 10]:
        # 計算簡單平均
        平均[f'{i}年平均盈餘分配率'] = sum(df['盈餘分配率'][:i]) / i
        
        # 計算加權平均
        try:
            W = sum(range(i+1))
            w = np.array([range(1, i+1)]) / W
            result = np.array(df['盈餘分配率'][:i]) * w
            加權[f'{i}年加權平均盈餘分配率'] = sum(result[0])
        except:
            i = len(df)
            W = sum(range(i+1))
            w = np.array([range(1, i+1)]) / W
            result = np.array(df['盈餘分配率'][:i]) * w
            加權[f'{len(df)}年加權平均盈餘分配率'] = sum(result[0])

    # === 抓取股價資訊 ===
    # 設定新的瀏覽器選項
    options = ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # 抓取股價和名稱
    driver = uc.Chrome(options=options)
    driver.get(f'https://www.cnyes.com/twstock/{代碼}/summary/overview')
    time.sleep(5)
    
    # 解析網頁內容
    page_source = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(page_source, 'html.parser')
    data = soup.find_all('div', id='anue-ga-wrapper')
    
    # 取得股價和名稱
    for i in data:
        stock = i.find_all('div', class_='jsx-2312976322 price')
        name = i.find_all('div', class_='jsx-2312976322 container')
        for i in stock:
            price = i.find_all('h3', class_='jsx-2312976322')
            for i in price:  
                股價 = float(i.text)
        for i in name:
            name = i.find_all('h2', class_='jsx-2312976322')
            for j in name:  
                股票名稱 = j.text

    # === 取得預估EPS ===
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2.5);")
    time.sleep(3)
    page_source = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(page_source, 'html.parser')
    data = soup.find_all('div', id='anue-ga-wrapper')
    
    預估eps = 0
    for i in data:
        info = i.find_all('div', id='anue-recent-wrapper')
        for j in info:
            eps = j.find_all('p', class_='jsx-1879398317')
            for m in eps:
                print(m.text)

    for ep in eps:
        text = ep.get_text()
        match = re.search(fr'{str(int(str(datetime.now().year)[2:])+1)}年\s+(\d+\.\d+)\s+元', text)
        if match:
            預估eps = float(match.group(1))
        else:
            print('沒有法人預估')
            預估eps = 0
    
    driver.quit()
    time.sleep(2)

    # === 計算預估值 ===
    資訊 = {}
    價位 = {}
    for i in [3, 5, 10]:
        try:
            # 計算預估殖利率和價位
            資訊[f'預估殖利率_{i}年'] = round(((預估eps * 平均[f'{i}年平均盈餘分配率']) / 股價) * 100, 2)
            價位[f'預估價位_{i}年平均'] = (預估eps * 平均[f'{i}年平均盈餘分配率']) / (歷年平均殖利率/100)
            
            資訊[f'預估殖利率_{i}年加權'] = round(((預估eps * 加權[f'{i}年加權平均盈餘分配率']) / 股價) * 100, 2)
            價位[f'預估價位_{i}年加權平均'] = (預估eps * 加權[f'{i}年加權平均盈餘分配率']) / (歷年平均殖利率/100)
        except:
            # 處理異常情況
            資訊[f'預估殖利率_{len(df)}年'] = round(((預估eps * 平均[f'{i}年平均盈餘分配率']) / 股價) * 100, 2)
            價位[f'預估價位_{len(df)}年平均'] = (預估eps * 平均[f'{i}年平均盈餘分配率']) / (歷年平均殖利率/100)
            
            資訊[f'預估殖利率_{len(df)}年加權'] = round(((預估eps * 加權[f'{len(df)}年加權平均盈餘分配率']) / 股價) * 100, 2)
            價位[f'預估價位_{len(df)}年加權平均'] = (預估eps * 加權[f'{len(df)}年加權平均盈餘分配率']) / (歷年平均殖利率/100)

    # === 輸出結果 ===
    try:
        print(f'{股票名稱}, \
        \n現在股價:{股價}, \
        \n歷年平均殖利率:{歷年平均殖利率}, \
        \n三年預估殖利率: {資訊["預估殖利率_3年"]}, \
        \n五年預估殖利率: {資訊["預估殖利率_5年"]}, \
        \n十年預估殖利率: {資訊["預估殖利率_10年"]}, \
        \n三年加權預估殖利率: {資訊["預估殖利率_3年加權"]}, \
        \n五年加權預估殖利率: {資訊["預估殖利率_5年加權"]}, \
        \n十年加權預估殖利率: {資訊["預估殖利率_10年加權"]}, \
        \n預估價位區間: {int(min(價位.values()))} ~ {int(max(價位.values()))}')
    except:
        print(f'{股票名稱}, \
        \n現在股價:{股價}, \
        \n歷年平均殖利率:{歷年平均殖利率}, \
        \n預估殖利率: {資訊}, \
        \n預估價位區間: {int(min(價位.values()))} ~ {int(max(價位.values()))}')

In [8]:
# === 輸入想要預估的標的代碼 ===
代碼 = int(input("輸入股票代碼："))
代碼

輸入股票代碼： 2454


2454

In [9]:
# === 開始抓取資料並計算 ===
getinfor(代碼)

python3.9(1165) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1166) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1167) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1168) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1169) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1170) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1172) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1173) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1174) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.9(1175) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


FactSet 對於近年的預估EPS做出預估中位數：23年 48.51 元，24年 65.91 元，25年 72.20 元，26年 84.24 元
聯發科2454上市,         
現在股價:1420.0,         
歷年平均殖利率:4.04,         
三年預估殖利率: 4.67,         
五年預估殖利率: 5.29,         
十年預估殖利率: 4.34,         
三年加權預估殖利率: 4.57,         
五年加權預估殖利率: 5.63,         
十年加權預估殖利率: 3.98,         
預估價位區間: 1399 ~ 1978
